In [7]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm
import time
import random
import string
import matplotlib.pyplot as plt

In [8]:
def stamp():

    timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    randomstamp = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6))
    stamp = timestamp + "_" + randomstamp
    return stamp

In [9]:
def envirement(reactiondistance):
    positions=[]
    posenzyme = [0,0,0]
    amountenzyme = 0
    while amountenzyme < 1000:
        #todo periodic boundaryconditions !!!!
        pos = np.random.random(3) * 19.999 - 9.999
        distance=np.sqrt((posenzyme[0]-pos[0])**2+(posenzyme[1]-pos[1])**2+(posenzyme[2]-pos[2])**2)
        if distance > reactiondistance:
            amountenzyme+=1
            positions.append(pos)
    positions=np.array(positions)
    return positions,posenzyme

 Simulation of 1500 particles with Diffusionconstant = 3, $\alpha=0.5 $ , $3 D t^\alpha$, Boxsize = 20, particle number =1500. The observable is the MSD of all particles. No reactions.
 

In [10]:

def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim("FractionalDiffusion")
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 0.01# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()

    sim.delete_all_particles()
    np.random.seed()

    for _ in range(1500):
         pos = np.random.random(3) * 9.999 - 4.999
         sim.add_particle(pos, 0)

    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"mean_e_test1.h5"
    sim.new_mean_squared_displacement(1,mean_e_name, 0)
    sim.run(6000)
    sim.write_observables_to_file()
    sim.delete_all_observables()

    fc = h5.File(mean_e_name, 'r')
    return  fc

In [11]:
def msd_analyt(alpha,dt,t,diffconst):
    t=np.linspace(0,t,t)
    return 2*3*((t*dt)**alpha)*diffconst

In [12]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
msd_text='MSD=2 K_\\alpha t^\\alpha'
for i in [3.0]:
    boxsize=20.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=3
    intrinsikreaction=10000000000000000000000
    colornow=next(colors)
    falist=[]
    fclist=[]
    averagelist=[]
    for ii in range(1):
        fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        fclist.append(fc['meanSquaredDisplacements'])
    fclist=np.array(fclist)  
    msd=fclist.mean(axis=0) 
    
    plt.errorbar(fc['times'], msd ,yerr=fc['standardErrors'],color=colornow,label="$D_S$=$%.2f$" %(i))
    plt.loglog(np.array(range(6000))*0.01,msd_analyt(0.5,0.01,6000,3.0),label="$%s$  " %(msd_text),linestyle='--')
    
plt.ylabel('MSD)', fontsize=10)
plt.xlabel('t', fontsize=10)
plt.legend()
plt.savefig('finalreport/data/fractionalmsdrecreaddy.png',dpi=300)   # save the figure to file 
plt.show()


Number of particles:  1500
TypeId | Number of particles
====== | ===================
0      | 1500               
1      | 0                  
2      | 0                  
 


Radial distribution function no reactions with fractional and normal borwnian motion

In [22]:
def simulation1(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()

    sim.delete_all_particles()
    np.random.seed()

    for _ in range(500):
         pos = np.random.random(3) * 19.999 - 9.999
         sim.add_particle(pos, 0)

    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"radialdist_e_test1.h5"
    sim.new_radial_distribution(1,mean_e_name,np.arange(0., 10., 0.1),np.array([[0,0]]))
    sim.run(6000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fc = h5.File(mean_e_name, 'r')
    return  fc

In [23]:
def simulation2(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim("FractionalDiffusion")
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.delete_all_particles()
    for _ in range(500):
         pos = np.random.random(3) * 19.999 - 9.999
         sim.add_particle(pos, 0)

    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"radialdist_e_test1.h5"
    sim.new_radial_distribution(1,mean_e_name,np.arange(0., 10., 0.1),np.array([[0,0]]))
    sim.run(6000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fc = h5.File(mean_e_name, 'r')
    return  fc

In [28]:
a =np.array([[1,1],[2,3]])
print len(a.shape)
print s

2


NameError: name 'traj' is not defined

In [25]:
fc=simulation1(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
fd=simulation2(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc

Number of particles:  500
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 0                  
 
Number of particles:  500
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 0                  
 


In [26]:
a=fc['binCenters']
plt.plot(fd['binCenters'],fc['bins'], label="fractional")
plt.plot(fc['binCenters'],fd['bins'], label="brownian")
plt.ylabel('Bins', fontsize=10)
plt.xlabel('Bin-Centers', fontsize=10)
plt.legend()
plt.savefig('finalreport/data/fractionalradialdistributionrevreaddy.png',dpi=300)   # save the figure to file 
plt.show()


Plan to generate a Film

In [ ]:
sim.new_trajectory